# Interpolation visualization

This documents shows effects of different approaches to the signal interpolation

In [ ]:
from functools import partial
from multiprocessing import cpu_count
from multiprocessing.pool import Pool

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from distance_determination import estimate_dist, get_current_freq
from interpolation.interpolate import interpolate, sklearn_interpolations
from interpolation.metrics import get_RMSE
from interpolation.wsinterp import wsinterp
from simul.signals.augment import signal_add_noise
from simul.utilities.data import load_experiment
from simul.vis.dist_probs import vis_dist_probs
from simul.vis.signals import get_vis_df, vis_signals, vis_signals2d, get_fft_df
from tqdm.auto import tqdm, trange

%load_ext autoreload
%autoreload 2

In [ ]:
from distance_determination import estimate_dist, simulate_signals
from run_experiment import experiments

# import plotly.io as pio
# pio.renderers.default = "notebook_connected"

# exp_name = "default_full"
exp_name = "full_no_walls"
params = experiments[exp_name]
dist, signals_data = simulate_signals(params)

# exp_name = "default"
exp_name = "no_walls"
params = experiments[exp_name]
dist, signals_data_pruned = simulate_signals(params)

Generate the data using different interpolations

In [ ]:
# interpolation_kinds = sklearn_interpolations + ["Whittaker–Shannon"]
interpolation_kinds = ["Whittaker–Shannon","randomized_sinc", ]
interp_signals = [
    (interpolate(signals_data_pruned, kind), kind) for kind in interpolation_kinds
]

The figure below shows the original signal, as well as different approaches to the interpolation. On the right you can toggle on or off the signals to have a better view.

In [ ]:
df = get_vis_df(
    params.tss,
    signals_data,
    signals_data_pruned,
    *interp_signals,
    n=20000,
    freqs=[0],
)

In [ ]:
vis_signals(df)

The above graph looks fancy, but a little bit messy. Take a look at the same graph in 2d.

In [ ]:
vis_signals2d(df)

In [ ]:
vis_signals2d(df, "imaginary")

Here will be an FFT comparison for different interpolations

In [ ]:
fft_df = get_fft_df(
    params.tss,
    signals_data,
    signals_data_pruned,
    (signals_data_pruned,"pruned"),
    *interp_signals,
    n = 20000,
    freqs=[0],
)
vis_signals2d(fft_df)

Interpolation with noise

In [ ]:
signals_pruned_noised = signal_add_noise(signals_data_pruned, val=0.0003)
interp_signals_noised = [
    (interpolate(signals_pruned_noised, kind), kind) for kind in interpolation_kinds
]

In [ ]:
df_noised = get_vis_df(
    params.tss,
    signals_data,
    signals_pruned_noised,
    (signals_pruned_noised,"measures_noised"),
    *interp_signals_noised,
    n=1000,
    freqs=[0],
)
vis_signals2d(df_noised)

In [ ]:
fft_df_noised = get_fft_df(
    params.tss,
    signals_data,
    signals_data_pruned,
    *interp_signals_noised,
    n = 20000,
    freqs=[0],
)
vis_signals2d(fft_df_noised)

Below the table with RMSEs for absolute value errors presented for different interpolations.

In [ ]:
df = get_vis_df(params.tss, signals_data, signals_data_pruned, *interp_signals)

fft_df = get_fft_df(
    params.tss,
    signals_data,
    signals_data_pruned,
    *interp_signals,
    n = 20000,
)

df_noised = get_vis_df(
    params.tss,
    signals_data,
    signals_pruned_noised,
    *interp_signals_noised,
)

fft_df_noised = get_fft_df(
    params.tss,
    signals_data,
    signals_data_pruned,
    *interp_signals_noised,
    n = 20000,
)


In [ ]:
rmse = get_RMSE(df)

rmse
# With wall:
#Whittaker–Shannon 	cubic 		linear 		nearest 	next 		previous 	quadratic
#0.007374 			0.004195 	0.011731 	0.022209 	0.041332 	0.040989 	0.004816

In [ ]:
rmse_noised = get_RMSE(df_noised)
rmse_noised

# With wall:
#Whittaker–Shannon 	cubic 		linear	 	nearest 	next 		previous 	quadratic
#0.018513 			0.016351 	0.018111 	0.027932 	0.044675 	0.044372 	0.016424

In [ ]:
# rmse_fft = get_RMSE(fft_df)
# rmse_fft


In [ ]:
# rmse_noised_fft = get_RMSE(fft_df_noised)
# rmse_noised_fft

The units are the same as are in the signal data

In [ ]:
f = partial(estimate_dist, params=params)

gt_dist_probs = f(signals_data)

with Pool(cpu_count()) as pool:
    args = [d[0] for d in interp_signals]
    dist_probss = pool.map(f, args)

In [ ]:
diffs = [dist_probs - gt_dist_probs for dist_probs in dist_probss]

dist_rmse = [
    np.sqrt(((np.real(diff) ** 2 + np.imag(diff) ** 2) ** 2).mean()) for diff in diffs
]

pd.DataFrame(
    data=dist_rmse, index=[name for signal, name in interp_signals], columns=["RMSE"]
).T

# With wall:
#linear 	nearest 	quadratic 	cubic 		previous 	next 		Whittaker–Shannon
#0.01563 	0.018498 	0.00975 	0.009659 	0.019834 	0.020572 	0.005389

In [ ]:
f = partial(estimate_dist, params=params)

with Pool(cpu_count()) as pool:
    args = [d[0] for d in interp_signals_noised]
    dist_probss_noised = pool.map(f, args)

In [ ]:
diffs_noised = [dist_probs - gt_dist_probs for dist_probs in dist_probss_noised]

dist_rmse = [
    np.sqrt(((np.real(diff) ** 2 + np.imag(diff) ** 2) ** 2).mean()) for diff in diffs_noised
]

pd.DataFrame(
    data=dist_rmse, index=[name for signal, name in interp_signals], columns=["RMSE"]
).T


# linear 	nearest 	quadratic 	cubic		previous 	next 		Whittaker–Shannon
# 0.011296 	0.023034 	0.009796 	0.009683	0.022723 	0.023808 	0.003016

# With wall:
# linear 	nearest 	quadratic 	cubic 		previous 	next 		Whittaker–Shannon
# 0.035209 	0.044421 	0.035234 	0.035442 	0.045815 	0.045743 	0.038139

In [ ]:
names = (name for _,name in interp_signals)
for name, dist_probs in zip(names, dist_probss_noised):
    print(name)
    vis_dist_probs(dist_probs, dist).update_layout(title=name).update_xaxes(title="Distance, m").update_yaxes(title="Predicted distance, m").show()


In [ ]:
vis_dist_probs(gt_dist_probs, dist).update_layout(title="Ideal").update_xaxes(title="Distance, m").update_yaxes(title="Predicted distance, m").show()